# 1. Importing the data
First, I'll import the libraries needed for the transformation

In [77]:
import pandas as pd 
from ast import literal_eval
import os

Then I'll import the csv files into a pandas dataframe

In [78]:
PATH = os.path.join('data')

raw_titles_df = pd.read_csv(os.path.join(PATH, 'raw_titles.csv'))
raw_credits_df = pd.read_csv(os.path.join(PATH, 'raw_credits.csv'))
best_movies_df = pd.read_csv(os.path.join(PATH, 'Best Movies Netflix.csv'))
best_movie_years_df = pd.read_csv(os.path.join(PATH, 'Best Movie by Year Netflix.csv'))
best_shows_df = pd.read_csv(os.path.join(PATH, 'Best Shows Netflix.csv'))
best_show_years_df = pd.read_csv(os.path.join(PATH, 'Best Show by Year Netflix.csv'))
print (best_show_years_df.head())

   index                         TITLE  RELEASE_YEAR  SCORE  \
0      0  Monty Python's Flying Circus          1969    8.8   
1      1                  Knight Rider          1982    6.9   
2      2                      Seinfeld          1989    8.9   
3      3    Star Trek: Deep Space Nine          1993    8.1   
4      4       Neon Genesis Evangelion          1995    8.5   

   NUMBER_OF_SEASONS MAIN_GENRE MAIN_PRODUCTION  
0                  4     comedy              GB  
1                  4     action              US  
2                  9     comedy              US  
3                  7      scifi              US  
4                  1      scifi              JP  


# 2. Transforming the data
Now I'll check every columns to see if there is anything wrong with the data.

In [79]:
cleaned_df = [] # Creating a list where I'll put the cleaned data.

## 2.1 Raw titles dataframe

In [80]:
print(raw_titles_df['id'].head(10))
print(raw_titles_df['id'].is_unique)

0    ts300399
1     tm84618
2    tm127384
3     tm70993
4    tm190788
5     ts22164
6     tm14873
7    tm185072
8     tm98978
9    tm119281
Name: id, dtype: object
True


We can see that the id column doesn't contains duplicates.   
We also see that the id column is a mix of characters and number and thus should stay as a string type.


In [81]:
print(raw_titles_df['title'].is_unique)
print(raw_titles_df['title'].head())

False
0    Five Came Back: The Reference Films
1                            Taxi Driver
2        Monty Python and the Holy Grail
3                          Life of Brian
4                           The Exorcist
Name: title, dtype: object


In [82]:
df = raw_titles_df.copy()
df['duplicated?'] = df['title'].duplicated(keep=False)
duplicated_df = df[df['duplicated?']]
print(duplicated_df.sort_values(by='title'))

      index        id                      title   type  release_year  \
657     657   ts19384        A Lion in the House   SHOW          2006   
519     519   tm25842        A Lion in the House  MOVIE          2006   
54       54    tm2386  A Nightmare on Elm Street  MOVIE          1984   
355     355  tm102337  A Nightmare on Elm Street  MOVIE          2010   
1731   1731  tm232710            A Second Chance  MOVIE          2015   
...     ...       ...                        ...    ...           ...   
38       38  tm259684                   Whispers  MOVIE          1980   
2258   2258  tm365824                       Zero  MOVIE          2018   
5441   5441  ts282155                       Zero   SHOW          2021   
1332   1332   ts35354                        Zoo   SHOW          2015   
2708   2708  tm355941                        Zoo  MOVIE          2018   

     age_certification  runtime  \
657              TV-14      225   
519                NaN      225   
54                

One row contain a Nan inside the title column, I'll remove it because it doesn't help to keep a movie where we don't know the title.

In [118]:
print(len(raw_titles_df))
raw_titles_df = raw_titles_df.dropna(subset=['title'])
print(len(raw_titles_df))

5806
5805


We can see there are some duplicates inside the title column but they refer to different movies/shows.

In [83]:
print(raw_titles_df['type'].unique())

['SHOW' 'MOVIE']


The type column correctly contains only 'MOVIE' or 'SHOW'

In [84]:
print (raw_titles_df['release_year'].head(10))

0    1945
1    1976
2    1975
3    1979
4    1973
5    1969
6    1971
7    1964
8    1980
9    1967
Name: release_year, dtype: int64


The 'release_year' data type is an integer but it will be converted to a date when importing the data to the RDBMS.

In [85]:
print(raw_titles_df['age_certification'].unique())

['TV-MA' 'R' 'PG' 'TV-14' 'G' 'PG-13' nan 'TV-PG' 'TV-Y' 'TV-G' 'TV-Y7'
 'NC-17']


The 'age_certification' column contain a rating or NaN.

In [86]:
print(raw_titles_df['runtime'].head(10))

0     48
1    113
2     91
3     94
4    133
5     30
6    102
7    170
8    104
9    110
Name: runtime, dtype: int64


The 'runtime' column is already as in integer.

In [87]:
print(raw_titles_df['genres'].head(10))

0                          ['documentation']
1                         ['crime', 'drama']
2                      ['comedy', 'fantasy']
3                                 ['comedy']
4                                 ['horror']
5                     ['comedy', 'european']
6            ['thriller', 'crime', 'action']
7    ['drama', 'music', 'romance', 'family']
8                       ['romance', 'drama']
9               ['drama', 'crime', 'action']
Name: genres, dtype: object


The 'genres' column contains lists of values. To normalize the data we want to split the elements inside the list to the movies.

In [88]:
genres_df = pd.DataFrame()
raw_titles_df['genres'] = raw_titles_df['genres'].apply(literal_eval) # Transforming the list into a correct format.
genres_df[['id','genres']] = raw_titles_df[['id','genres']].explode(column='genres')

print(genres_df.head(10))
raw_titles_df = raw_titles_df.drop(columns='genres') # Dropping the column because the data is in another dataframe.
cleaned_df.append(genres_df)

         id         genres
0  ts300399  documentation
1   tm84618          crime
1   tm84618          drama
2  tm127384         comedy
2  tm127384        fantasy
3   tm70993         comedy
4  tm190788         horror
5   ts22164         comedy
5   ts22164       european
6   tm14873       thriller


In [89]:
print (raw_titles_df['production_countries'].head(20))

0           ['US']
1           ['US']
2           ['GB']
3           ['GB']
4           ['US']
5           ['GB']
6           ['US']
7           ['US']
8           ['US']
9           ['US']
10          ['US']
11    ['US', 'GB']
12          ['JP']
13          ['US']
14          ['US']
15          ['US']
16          ['EG']
17          ['US']
18          ['DE']
19          ['US']
Name: production_countries, dtype: object


Similarly to the genre column, we want to split the lists inside the 'production_countries' columns.

In [90]:
production_df = pd.DataFrame()
raw_titles_df['production_countries'] = raw_titles_df['production_countries'].apply(literal_eval) # Transforming the list into a correct format.
production_df[['id','production_countries']] = raw_titles_df[['id','production_countries']].explode(column='production_countries')

cleaned_df.append(production_df)
raw_titles_df = raw_titles_df.drop(columns='production_countries')
print(production_df.head(10))

         id production_countries
0  ts300399                   US
1   tm84618                   US
2  tm127384                   GB
3   tm70993                   GB
4  tm190788                   US
5   ts22164                   GB
6   tm14873                   US
7  tm185072                   US
8   tm98978                   US
9  tm119281                   US


In [91]:
print(raw_titles_df['seasons'].value_counts())

seasons
1.0     1187
2.0      374
3.0      181
4.0      116
5.0       76
6.0       40
7.0       16
8.0       14
9.0        9
11.0       7
10.0       5
15.0       4
12.0       4
24.0       3
13.0       2
14.0       2
19.0       1
18.0       1
29.0       1
21.0       1
42.0       1
39.0       1
37.0       1
Name: count, dtype: int64


The seasons column has either a NaN or the number of seasons for the show.

In [92]:
print(raw_titles_df['imdb_id'].is_unique)

False


In [93]:
df = raw_titles_df.copy()
df['duplicated?'] = df['imdb_id'].duplicated(keep=False)
duplicated_df = df[df['duplicated?']]
print(duplicated_df[['id','title','type','release_year','imdb_id']].sort_values(by='imdb_id'))

             id                                      title   type  \
0      ts300399        Five Came Back: The Reference Films   SHOW   
32      tm97793                         Lock Your Girls In  MOVIE   
33      tm85426        The Witness Who Didn't See Anything  MOVIE   
34     tm188894                     The School of Mischief  MOVIE   
91     tm126791                        The Married Couples  MOVIE   
...         ...                                        ...    ...   
5780  tm1074617              Bling Empire - The Afterparty  MOVIE   
5783  tm1035612  Dad Stop Embarrassing Me - The Afterparty  MOVIE   
5794   ts297061                            Masameer County   SHOW   
5799   ts307884                                 HQ Barbers   SHOW   
5802  tm1108171                             Edis Starlight  MOVIE   

      release_year imdb_id  
0             1945     NaN  
32            1980     NaN  
33            1976     NaN  
34            1973     NaN  
91            1981     NaN

The IMDB_id column contains duplicates but those are NaN.

In [94]:
print(raw_titles_df['imdb_score'].head(10))
print(raw_titles_df['imdb_votes'].head(10))

0    NaN
1    8.3
2    8.2
3    8.0
4    8.1
5    8.8
6    7.7
7    7.8
8    5.8
9    7.7
Name: imdb_score, dtype: float64
0         NaN
1    795222.0
2    530877.0
3    392419.0
4    391942.0
5     72895.0
6    153463.0
7     94121.0
8     69053.0
9    111189.0
Name: imdb_votes, dtype: float64


The IMDB_vote and IMDB_score columns are already in the correct format.

In [95]:
raw_titles_df = raw_titles_df.drop(columns='index')
print(raw_titles_df.head(10))
cleaned_df.append(raw_titles_df)

         id                                title   type  release_year  \
0  ts300399  Five Came Back: The Reference Films   SHOW          1945   
1   tm84618                          Taxi Driver  MOVIE          1976   
2  tm127384      Monty Python and the Holy Grail  MOVIE          1975   
3   tm70993                        Life of Brian  MOVIE          1979   
4  tm190788                         The Exorcist  MOVIE          1973   
5   ts22164         Monty Python's Flying Circus   SHOW          1969   
6   tm14873                          Dirty Harry  MOVIE          1971   
7  tm185072                         My Fair Lady  MOVIE          1964   
8   tm98978                      The Blue Lagoon  MOVIE          1980   
9  tm119281                     Bonnie and Clyde  MOVIE          1967   

  age_certification  runtime  seasons    imdb_id  imdb_score  imdb_votes  
0             TV-MA       48      1.0        NaN         NaN         NaN  
1                 R      113      NaN  tt00753

Lastly, I drop the index column and add the raw_title df to the cleaned_df list.

## 2.2 Raw credits dataframe

In [96]:
print(raw_credits_df.head(10))

   index  person_id       id             name                      character  \
0      0       3748  tm84618   Robert De Niro                  Travis Bickle   
1      1      14658  tm84618     Jodie Foster                  Iris Steensma   
2      2       7064  tm84618    Albert Brooks                            Tom   
3      3       3739  tm84618    Harvey Keitel        Matthew 'Sport' Higgins   
4      4      48933  tm84618  Cybill Shepherd                          Betsy   
5      5      32267  tm84618      Peter Boyle                         Wizard   
6      6     519612  tm84618   Leonard Harris      Senator Charles Palantine   
7      7      29068  tm84618   Diahnne Abbott                Concession Girl   
8      8     519613  tm84618      Gino Ardito             Policeman at Rally   
9      9       3308  tm84618  Martin Scorsese  Passenger Watching Silhouette   

    role  
0  ACTOR  
1  ACTOR  
2  ACTOR  
3  ACTOR  
4  ACTOR  
5  ACTOR  
6  ACTOR  
7  ACTOR  
8  ACTOR  
9  ACTOR 

The 'person_id' and 'id' columns refers to the id of the actor and the id of the movies. It's normal for those to have duplicates and they are in the correct format.
The 'name' and 'character' columns are also correctly formated already.

In [97]:
print(raw_credits_df['role'].value_counts())

role
ACTOR       72690
DIRECTOR     4523
Name: count, dtype: int64


The 'role' column contains either 'Actor' or 'Director' and doesn't have any incorrect data.

In [98]:
raw_credits_df = raw_credits_df.drop(columns='index')
cleaned_df.append(raw_credits_df)
print(raw_credits_df.head())

   person_id       id             name                character   role
0       3748  tm84618   Robert De Niro            Travis Bickle  ACTOR
1      14658  tm84618     Jodie Foster            Iris Steensma  ACTOR
2       7064  tm84618    Albert Brooks                      Tom  ACTOR
3       3739  tm84618    Harvey Keitel  Matthew 'Sport' Higgins  ACTOR
4      48933  tm84618  Cybill Shepherd                    Betsy  ACTOR


I remove the index column and add it to the cleaned_df list.

## 2.3 Best movies dataframe

In [99]:
print(best_movies_df.head(10))

   index                                     TITLE  RELEASE_YEAR  SCORE  \
0      0  David Attenborough: A Life on Our Planet          2020    9.0   
1      1                                 Inception          2010    8.8   
2      2                              Forrest Gump          1994    8.8   
3      3                                Anbe Sivam          2003    8.7   
4      4                        Bo Burnham: Inside          2021    8.7   
5      5                       Saving Private Ryan          1998    8.6   
6      6                          Django Unchained          2012    8.4   
7      7                                    Dangal          2016    8.4   
8      8                    Bo Burnham: Make Happy          2016    8.4   
9      9                     Louis C.K.: Hilarious          2010    8.4   

   NUMBER_OF_VOTES  DURATION   MAIN_GENRE MAIN_PRODUCTION  
0            31180        83  documentary              GB  
1          2268288       148        scifi             

In [100]:
print(best_movies_df['TITLE'].head())
print(best_movies_df['RELEASE_YEAR'].head())
print(best_movies_df['SCORE'].head())
print(best_movies_df['NUMBER_OF_VOTES'].head())
print(best_movies_df['DURATION'].head())

0    David Attenborough: A Life on Our Planet
1                                   Inception
2                                Forrest Gump
3                                  Anbe Sivam
4                          Bo Burnham: Inside
Name: TITLE, dtype: object
0    2020
1    2010
2    1994
3    2003
4    2021
Name: RELEASE_YEAR, dtype: int64
0    9.0
1    8.8
2    8.8
3    8.7
4    8.7
Name: SCORE, dtype: float64
0      31180
1    2268288
2    1994599
3      20595
4      44074
Name: NUMBER_OF_VOTES, dtype: int64
0     83
1    148
2    142
3    160
4     87
Name: DURATION, dtype: int64


In [101]:
print(best_movies_df['TITLE'].is_unique)

True


Those 5 columns are already in the correct format and there are no duplicated titles.

In [102]:
print(best_movies_df['MAIN_GENRE'].value_counts())
print(best_movies_df['MAIN_PRODUCTION'].value_counts())

MAIN_GENRE
drama          151
thriller        59
comedy          58
crime           21
romance         21
documentary     20
fantasy         19
horror           8
scifi            7
western          7
action           5
musical          4
war              3
animation        3
sports           1
Name: count, dtype: int64
MAIN_PRODUCTION
US    157
IN    115
GB     29
DE      9
JP      9
FR      7
TR      7
ES      6
IT      4
KR      4
CA      4
AU      3
CN      3
HK      2
NL      2
BE      2
NO      2
NZ      2
IE      2
HU      2
UA      2
KH      1
DK      1
LT      1
XX      1
BR      1
MX      1
PL      1
PS      1
AR      1
TH      1
MW      1
ZA      1
CD      1
ID      1
Name: count, dtype: int64


The 'MAIN_GENRES' and 'MAIN_PRODUCTION' contains only one entry and are formated correctly.

In [103]:
best_movies_df = best_movies_df.drop(columns='index')
print(best_movies_df.head())

                                      TITLE  RELEASE_YEAR  SCORE  \
0  David Attenborough: A Life on Our Planet          2020    9.0   
1                                 Inception          2010    8.8   
2                              Forrest Gump          1994    8.8   
3                                Anbe Sivam          2003    8.7   
4                        Bo Burnham: Inside          2021    8.7   

   NUMBER_OF_VOTES  DURATION   MAIN_GENRE MAIN_PRODUCTION  
0            31180        83  documentary              GB  
1          2268288       148        scifi              GB  
2          1994599       142        drama              US  
3            20595       160       comedy              IN  
4            44074        87       comedy              US  


We can remove the index column.

We want to add an ID column that link the movies to their id from the 'raw_titles' dataframe.

In [104]:
best_movies_df = best_movies_df.merge(raw_titles_df, left_on='TITLE', right_on='title', how='left')[['id','TITLE','RELEASE_YEAR','SCORE','NUMBER_OF_VOTES','DURATION','MAIN_GENRE','MAIN_PRODUCTION']]
print(best_movies_df.head())

          id                                     TITLE  RELEASE_YEAR  SCORE  \
0   tm853783  David Attenborough: A Life on Our Planet          2020    9.0   
1    tm92641                                 Inception          2010    8.8   
2   tm122434                              Forrest Gump          1994    8.8   
3   tm129763                                Anbe Sivam          2003    8.7   
4  tm1038686                        Bo Burnham: Inside          2021    8.7   

   NUMBER_OF_VOTES  DURATION   MAIN_GENRE MAIN_PRODUCTION  
0            31180        83  documentary              GB  
1          2268288       148        scifi              GB  
2          1994599       142        drama              US  
3            20595       160       comedy              IN  
4            44074        87       comedy              US  


In [105]:
cleaned_df.append(best_movies_df)

## 2.4 Best movies by years dataframe

In [106]:
print(best_movie_years_df.head())

   index                 TITLE  RELEASE_YEAR  SCORE MAIN_GENRE MAIN_PRODUCTION
0      0       White Christmas          1954    7.5    romance              US
1      1  The Guns of Navarone          1961    7.5        war              US
2      2          My Fair Lady          1964    7.8      drama              US
3      3      Bonnie and Clyde          1967    7.7      drama              US
4      4           Dirty Harry          1971    7.7   thriller              US


In [107]:
print(best_movie_years_df['TITLE'].is_unique)

True


Like the previous dataframe, the data is already correctly formated.
We just need to remove the index column and add the id column.

In [108]:
best_movie_years_df = best_movie_years_df.merge(raw_titles_df, left_on='TITLE', right_on='title', how='left')[['id','TITLE','RELEASE_YEAR','SCORE','MAIN_GENRE','MAIN_PRODUCTION']]
print(best_movies_df.head())

          id                                     TITLE  RELEASE_YEAR  SCORE  \
0   tm853783  David Attenborough: A Life on Our Planet          2020    9.0   
1    tm92641                                 Inception          2010    8.8   
2   tm122434                              Forrest Gump          1994    8.8   
3   tm129763                                Anbe Sivam          2003    8.7   
4  tm1038686                        Bo Burnham: Inside          2021    8.7   

   NUMBER_OF_VOTES  DURATION   MAIN_GENRE MAIN_PRODUCTION  
0            31180        83  documentary              GB  
1          2268288       148        scifi              GB  
2          1994599       142        drama              US  
3            20595       160       comedy              IN  
4            44074        87       comedy              US  


In [109]:
cleaned_df.append(best_movie_years_df)

## 2.5 Best show dataframe

In [110]:
print(best_shows_df.head(10))

   index                       TITLE  RELEASE_YEAR  SCORE  NUMBER_OF_VOTES  \
0      0                Breaking Bad          2008    9.5          1727694   
1      1  Avatar: The Last Airbender          2005    9.3           297336   
2      2                  Our Planet          2019    9.3            41386   
3      3                Kota Factory          2019    9.3            66985   
4      4              The Last Dance          2020    9.1           108321   
5      5                      Arcane          2021    9.1           175412   
6      6             Attack on Titan          2013    9.0           325381   
7      7             Hunter x Hunter          2011    9.0            87857   
8      8                  DEATH NOTE          2006    9.0           302147   
9      9                    Seinfeld          1989    8.9           302700   

   DURATION  NUMBER_OF_SEASONS   MAIN_GENRE MAIN_PRODUCTION  
0        48                  5        drama              US  
1        24      

In [111]:
print(best_shows_df['TITLE'].is_unique)

True


The data follows a similar structure than the best movies dataframe.
The columns are formated correctly and I'll just replace the index column by the column.

In [112]:
best_shows_df = best_shows_df.merge(raw_titles_df, left_on='TITLE', right_on='title', how='left')[['id','TITLE','RELEASE_YEAR','SCORE','NUMBER_OF_VOTES','DURATION','NUMBER_OF_SEASONS','MAIN_GENRE','MAIN_PRODUCTION']]
print(best_shows_df.head())

        id                       TITLE  RELEASE_YEAR  SCORE  NUMBER_OF_VOTES  \
0      ts4                Breaking Bad          2008    9.5          1727694   
1   ts3371  Avatar: The Last Airbender          2005    9.3           297336   
2  ts85398                  Our Planet          2019    9.3            41386   
3  ts90621                Kota Factory          2019    9.3            66985   
4  ts81120              The Last Dance          2020    9.1           108321   

   DURATION  NUMBER_OF_SEASONS   MAIN_GENRE MAIN_PRODUCTION  
0        48                  5        drama              US  
1        24                  3        scifi              US  
2        50                  1  documentary              GB  
3        42                  2        drama              IN  
4        50                  1  documentary              US  


In [113]:
cleaned_df.append(best_shows_df)

## 2.6 Best shows by years dataframe

In [114]:
print(best_show_years_df.head(10))

   index                         TITLE  RELEASE_YEAR  SCORE  \
0      0  Monty Python's Flying Circus          1969    8.8   
1      1                  Knight Rider          1982    6.9   
2      2                      Seinfeld          1989    8.9   
3      3    Star Trek: Deep Space Nine          1993    8.1   
4      4       Neon Genesis Evangelion          1995    8.5   
5      5                 Stargate SG-1          1997    8.4   
6      6                  Cowboy Bebop          1998    8.9   
7      7                     One Piece          1999    8.8   
8      8                 Gilmore Girls          2000    8.2   
9      9             Trailer Park Boys          2001    8.6   

   NUMBER_OF_SEASONS MAIN_GENRE MAIN_PRODUCTION  
0                  4     comedy              GB  
1                  4     action              US  
2                  9     comedy              US  
3                  7      scifi              US  
4                  1      scifi              JP  
5     

The data follows a similar structure than the best movies by years dataframe.
The columns are formated correctly and I'll just replace the index column by the column.

In [116]:
best_show_years_df = best_show_years_df.merge(raw_titles_df, left_on='TITLE', right_on='title', how='left')[['id','TITLE','RELEASE_YEAR','SCORE','NUMBER_OF_SEASONS','MAIN_GENRE','MAIN_PRODUCTION']]
print(best_show_years_df.head())

        id                         TITLE  RELEASE_YEAR  SCORE  \
0  ts22164  Monty Python's Flying Circus          1969    8.8   
1  ts22082                  Knight Rider          1982    6.9   
2  ts20681                      Seinfeld          1989    8.9   
3  ts20927    Star Trek: Deep Space Nine          1993    8.1   
4  ts20983       Neon Genesis Evangelion          1995    8.5   

   NUMBER_OF_SEASONS MAIN_GENRE MAIN_PRODUCTION  
0                  4     comedy              GB  
1                  4     action              US  
2                  9     comedy              US  
3                  7      scifi              US  
4                  1      scifi              JP  


In [117]:
cleaned_df.append(best_show_years_df)